# CUDA Kernel Development Guide

This notebook covers CUDA kernel development from basics to advanced fusion kernels.

## Development Path:
1. **CPU Prototyping** (MacBook Pro friendly)
2. **Simple CUDA Kernels** (Element-wise operations)
3. **Advanced Fusion Kernels** (Attention, LayerNorm, etc.)
4. **Optimization Techniques** (Memory coalescing, shared memory, etc.)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
else:
    print("Running on CPU - perfect for learning CUDA concepts!")

## 1. CPU Prototyping (MacBook Pro Friendly)

Start by implementing algorithms on CPU to understand the logic:

In [ ]:
def cpu_attention_prototype(q, k, v, mask=None):
    """
    CPU prototype of attention mechanism
    This helps understand the algorithm before writing CUDA
    """
    batch, heads, seq_len, d_head = q.shape
    scale = 1.0 / (d_head ** 0.5)
    
    # Step 1: Compute attention scores
    scores = torch.matmul(q, k.transpose(-2, -1)) * scale
    print(f"Attention scores shape: {scores.shape}")
    
    # Step 2: Apply mask (if provided)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))
    
    # Step 3: Softmax normalization
    attn_weights = torch.softmax(scores, dim=-1)
    print(f"Attention weights shape: {attn_weights.shape}")
    
    # Step 4: Apply attention to values
    output = torch.matmul(attn_weights, v)
    print(f"Output shape: {output.shape}")
    
    return output, attn_weights

# Test the prototype
batch, heads, seq_len, d_head = 2, 8, 64, 32
q = torch.randn(batch, heads, seq_len, d_head)
k = torch.randn(batch, heads, seq_len, d_head)
v = torch.randn(batch, heads, seq_len, d_head)

output, weights = cpu_attention_prototype(q, k, v)
print(f"\nSuccessfully computed attention on CPU!")

## 2. CUDA Kernel Concepts

### Key CUDA Concepts:
1. **Threads and Blocks**: Parallel execution units
2. **Memory Hierarchy**: Global, shared, registers
3. **Memory Coalescing**: Efficient memory access patterns
4. **Occupancy**: Maximizing GPU utilization

In [ ]:
# CUDA kernel structure (pseudocode for learning)
cuda_kernel_template = """
__global__ void my_kernel(
    const float* input,   // Input data
    float* output,        // Output data
    const int n           // Size
) {
    // 1. Calculate thread index
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // 2. Bounds checking
    if (idx < n) {
        // 3. Compute operation
        output[idx] = input[idx] * 2.0f;  // Example: scale by 2
    }
}
"""

print("Basic CUDA Kernel Structure:")
print(cuda_kernel_template)

# Explain memory hierarchy
print("\nCUDA Memory Hierarchy:")
print("1. Global Memory: Slow but large (GPU VRAM)")
print("2. Shared Memory: Fast, shared within block")
print("3. Registers: Fastest, per-thread private")
print("4. Constant Memory: Read-only, cached")

## 3. Simple CUDA Kernels (Learning Examples)

These examples show basic CUDA patterns without requiring GPU:

In [ ]:
# Element-wise addition kernel (CUDA pseudocode)
add_kernel_code = """
__global__ void add_kernel(
    const float* a,
    const float* b, 
    float* result,
    const int n
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = a[idx] + b[idx];
    }
}
"""

# ReLU activation kernel
relu_kernel_code = """
__global__ void relu_kernel(
    const float* input,
    float* output,
    const int n
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = fmaxf(0.0f, input[idx]);
    }
}
"""

print("Element-wise Addition Kernel:")
print(add_kernel_code)
print("\nReLU Activation Kernel:")
print(relu_kernel_code)

# CPU implementations for comparison
def cpu_add(a, b):
    return a + b

def cpu_relu(x):
    return torch.clamp(x, min=0)

# Test on sample data
a = torch.randn(1000)
b = torch.randn(1000)
x = torch.randn(1000)

result_add = cpu_add(a, b)
result_relu = cpu_relu(x)

print(f"\nCPU Add result shape: {result_add.shape}")
print(f"CPU ReLU result shape: {result_relu.shape}")
print(f"ReLU zeros negative values: {(result_relu >= 0).all()}")

## 4. Fused Attention Kernel Design

Break down the attention mechanism for CUDA implementation:

In [ ]:
def analyze_attention_computation(q, k, v):
    """
    Analyze attention computation for CUDA kernel design
    """
    batch, heads, seq_len, d_head = q.shape
    
    print(f"Input dimensions:")
    print(f"  Batch size: {batch}")
    print(f"  Number of heads: {heads}")
    print(f"  Sequence length: {seq_len}")
    print(f"  Head dimension: {d_head}")
    
    # Step 1: QK^T computation
    scores = torch.matmul(q, k.transpose(-2, -1))
    print(f"\nStep 1 - QK^T:")
    print(f"  Operation: {q.shape} × {k.transpose(-2, -1).shape} = {scores.shape}")
    print(f"  FLOPs per head: {seq_len} × {seq_len} × {d_head} = {seq_len * seq_len * d_head:,}")
    
    # Step 2: Scaling
    scale = 1.0 / (d_head ** 0.5)
    scores = scores * scale
    print(f"\nStep 2 - Scaling:")
    print(f"  Scale factor: {scale:.4f}")
    
    # Step 3: Softmax
    attn_weights = torch.softmax(scores, dim=-1)
    print(f"\nStep 3 - Softmax:")
    print(f"  Input shape: {scores.shape}")
    print(f"  Output shape: {attn_weights.shape}")
    
    # Step 4: Attention × Values
    output = torch.matmul(attn_weights, v)
    print(f"\nStep 4 - Attention × Values:")
    print(f"  Operation: {attn_weights.shape} × {v.shape} = {output.shape}")
    print(f"  FLOPs per head: {seq_len} × {seq_len} × {d_head} = {seq_len * seq_len * d_head:,}")
    
    # Memory analysis
    total_elements = batch * heads * seq_len * d_head
    intermediate_elements = batch * heads * seq_len * seq_len
    
    print(f"\nMemory Analysis:")
    print(f"  Input tensors (Q,K,V): {3 * total_elements * 4 / 1024**2:.2f} MB")
    print(f"  Attention matrix: {intermediate_elements * 4 / 1024**2:.2f} MB")
    print(f"  Output tensor: {total_elements * 4 / 1024**2:.2f} MB")
    
    return output

# Analyze a sample attention computation
q_sample = torch.randn(2, 8, 128, 64)
k_sample = torch.randn(2, 8, 128, 64)
v_sample = torch.randn(2, 8, 128, 64)

output = analyze_attention_computation(q_sample, k_sample, v_sample)

## 5. CUDA Kernel Optimization Strategies

In [ ]:
optimization_strategies = {
    "Memory Coalescing": {
        "description": "Access consecutive memory locations",
        "example": "Access data[idx] instead of data[idx * stride]",
        "benefit": "Up to 10x memory bandwidth improvement"
    },
    "Shared Memory": {
        "description": "Cache frequently accessed data in fast shared memory",
        "example": "Load tile of data into __shared__ memory",
        "benefit": "100x faster than global memory access"
    },
    "Occupancy Optimization": {
        "description": "Maximize threads per SM",
        "example": "Use 256 threads per block for modern GPUs",
        "benefit": "Better latency hiding"
    },
    "Loop Unrolling": {
        "description": "Reduce loop overhead",
        "example": "#pragma unroll for small loops",
        "benefit": "Reduced instruction overhead"
    },
    "Warp-level Primitives": {
        "description": "Use warp shuffle and reduce operations",
        "example": "__shfl_down_sync for reductions",
        "benefit": "Faster collective operations"
    }
}

print("CUDA Optimization Strategies:")
print("=" * 50)
for strategy, details in optimization_strategies.items():
    print(f"\n{strategy}:")
    print(f"  Description: {details['description']}")
    print(f"  Example: {details['example']}")
    print(f"  Benefit: {details['benefit']}")

## 6. Development Workflow

In [ ]:
development_workflow = [
    "1. Algorithm Design (CPU prototype)",
    "2. Naive CUDA Implementation", 
    "3. Correctness Testing",
    "4. Performance Profiling",
    "5. Memory Optimization",
    "6. Compute Optimization",
    "7. Final Validation"
]

print("CUDA Kernel Development Workflow:")
print("=" * 40)
for step in development_workflow:
    print(step)

print("\nTools for Each Step:")
tools = {
    "Prototyping": "PyTorch CPU implementations",
    "Development": "PyTorch C++ extensions, NVCC", 
    "Testing": "torch.allclose(), unit tests",
    "Profiling": "nsys, ncu, PyTorch profiler",
    "Debugging": "cuda-gdb, compute-sanitizer"
}

for tool_type, tool_name in tools.items():
    print(f"  {tool_type}: {tool_name}")

## 7. Next Steps

### For MacBook Pro Development:
1. **Master the concepts** using this CPU-based environment
2. **Design algorithms** and test correctness
3. **Write pseudo-CUDA code** to understand the patterns

### For GPU Development:
1. **Set up cloud GPU** instance (AWS, Google Cloud, etc.)
2. **Implement actual CUDA kernels** using the patterns learned
3. **Profile and optimize** using GPU tools

### Learning Resources:
- NVIDIA CUDA Programming Guide
- "Programming Massively Parallel Processors" book
- CUDA samples and documentation
- PyTorch extension tutorials

In [ ]:
# Summary of what we've covered
print("🎯 Summary - You Now Know:")
print("=" * 30)
print("✅ CUDA kernel structure and concepts")
print("✅ Memory hierarchy and optimization")
print("✅ Attention mechanism breakdown")
print("✅ Development workflow and tools")
print("✅ Performance optimization strategies")
print("")
print("🚀 Ready for GPU Development!")
print("   Set up a cloud GPU when you're ready to implement real CUDA kernels.")